In [1]:
!git clone https://github.com/acsicuib/YAFS.git

Cloning into 'YAFS'...
remote: Enumerating objects: 3556, done.
remote: Counting objects: 100% (551/551), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 3556 (delta 425), reused 417 (delta 417), pack-reused 3005 (from 1)
Receiving objects: 100% (3556/3556), 104.90 MiB | 10.39 MiB/s, done.
Resolving deltas: 100% (1881/1881), done.
Updating files: 100% (2391/2391), done.


In [2]:
%cd YAFS
!pip install .

/content/YAFS
Processing /content/YAFS
  Preparing metadata (setup.py) ... done
  Created wheel for yafs: filename=yafs-0.3.0-py2.py3-none-any.whl size=32539 sha256=ad97639723ea8f82a6b389724360d8e0ca6ec08c3b734cd5966e54a99378b045
  Stored in directory: /tmp/pip-ephem-wheel-cache-pb9mt0s8/wheels/e9/d0/5c/197bbba407a3c39bbd2b144f18df5066abc456673b4c5b0659
Successfully built yafs


In [3]:
import yafs

In [8]:
import yafs.selection

# List all attributes in the placement module
print(dir(yafs.selection))

['First_ShortestPath', 'OneRandomPath', 'Selection', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'logging', 'nx', 'random']


In [9]:
from yafs.core import Sim
from yafs.topology import Topology
from yafs.application import Application, Message
from yafs.placement import EdgePlacement
from yafs.selection import First_ShortestPath
from yafs.distribution import deterministic_distribution

In [17]:
import networkx as nx
from yafs.topology import Topology

# Initialize the topology and create a directed graph
topology = Topology()
topology.G = nx.DiGraph()  # Use a directed graph

# Add Cloud node (one centralized node)
cloud_id = "Cloud"
topology.G.add_node(cloud_id, RAM=10000, IPT=1000)  # IPT: Instructions per time unit

# Add Fog nodes (three in this example) and link them to the Cloud
fog_ids = []
for i in range(3):
    fog_id = f"Fog_{i}"
    topology.G.add_node(fog_id, RAM=4000, IPT=500)
    topology.G.add_edge(fog_id, cloud_id, BW=1000, PR=10)  # Link fog to cloud
    fog_ids.append(fog_id)

print("Cloud and fog nodes added to the topology.")


Cloud and fog nodes added to the topology.


In [18]:
# Add End Devices (IoT sensors) and connect them to the Fog nodes
device_types = ["HeartRateSensor", "TemperatureSensor", "PulseOximeter", "ECGSensor", "Laptop", "Camera"]
device_ids = []
for i, device in enumerate(device_types):
    device_id = f"{device}_{i}"
    topology.G.add_node(device_id, RAM=500, IPT=50)
    topology.G.add_edge(device_id, fog_ids[i % 3], BW=100, PR=20)  # Link device to one of the fog nodes
    device_ids.append(device_id)

print("End devices added to the topology.")


End devices added to the topology.


In [21]:
# Define Messages (data flow)
# Message from devices to fog nodes
msg_device_to_fog = Message("DeviceToFog", "DATA", fog_ids)

# Message from fog nodes to the cloud
msg_fog_to_cloud = Message("FogToCloud", "DATA", [cloud_id])

print("Application and messages defined.")


Application and messages defined.


In [31]:
from yafs.application import Application

# Define the application structure as a dictionary
app_config = {
    "name": "IoT_App",
    "modules": [
        {"id": "DeviceModule", "type": Application.TYPE_SOURCE, "RAM": 500},
        {"id": "FogModule", "type": Application.TYPE_MODULE, "RAM": 2000},
        {"id": "CloudModule", "type": Application.TYPE_SINK, "RAM": 10000}
    ],
    "messages": [
        {"id": "msg_device_to_fog", "s_module": "DeviceModule", "d_module": "FogModule", "instructions": 1000, "bytes": 500},
        {"id": "msg_fog_to_cloud", "s_module": "FogModule", "d_module": "CloudModule", "instructions": 5000, "bytes": 1500}
    ],
    "services": [
        {"id": "service_processing", "module": "FogModule", "type": Application.TYPE_MODULE, "RAM": 2000}
    ]
}

# Initialize the Application with the configuration
application = Application(app_config)

print("Application defined with modules, messages, and services.")


Application defined with modules, messages, and services.


In [32]:
from yafs.placement import EdgePlacement
from yafs.selection import First_ShortestPath

# Define Placement and Selection Policies
placement = EdgePlacement(name="EdgePlacement")
selection = First_ShortestPath()

print("Placement and selection policies defined.")


Placement and selection policies defined.


In [33]:
from yafs.core import Sim

# Initialize the simulation with the topology
sim = Sim(topology)

# Deploy the application with the placement and selection policies
sim.deploy_app(application, placement, selection)  # Pass arguments positionally

# Run the simulation
sim.run(stop_time=1000)  # Run for 1000 time units

print("Simulation completed.")


TypeError: unhashable type: 'dict'

In [34]:
# Initialize the topology and create a directed graph
topology = Topology()
topology.G = nx.DiGraph()  # Directed graph

# Add Cloud node
cloud_id = "Cloud"
topology.G.add_node(cloud_id, RAM=10000, IPT=1000)

# Add Fog nodes and link to Cloud
fog_ids = []
for i in range(3):
    fog_id = f"Fog_{i}"
    topology.G.add_node(fog_id, RAM=4000, IPT=500)
    topology.G.add_edge(fog_id, cloud_id, BW=1000, PR=10)
    fog_ids.append(fog_id)

# Add IoT devices and link to Fog nodes
device_ids = []
for i in range(6):
    device_id = f"Device_{i}"
    topology.G.add_node(device_id, RAM=500, IPT=50)
    topology.G.add_edge(device_id, fog_ids[i % len(fog_ids)], BW=100, PR=20)
    device_ids.append(device_id)

print("Topology defined with cloud, fog nodes, and devices.")


Topology defined with cloud, fog nodes, and devices.


In [35]:
from yafs.application import Application

# Create a basic application with a simple name
application = Application(name="MinimalApp")

# Assuming no complex module/message setup required for this example
print("Application initialized as 'MinimalApp'.")


Application initialized as 'MinimalApp'.


In [36]:
# Set up placement and selection policies
placement = EdgePlacement(name="EdgePlacement")
selection = First_ShortestPath()

print("Placement and selection policies defined.")


Placement and selection policies defined.


In [38]:
# Initialize the simulation
sim = Sim(topology)

# Deploy the application
sim.deploy_app(application, placement, selection)

# Run the simulation for a basic time period (100 time units)
sim.run(100)

print("Minimal simulation completed.")


----------------------------------------
DES	| TOPO 	| Src.Mod 	| Modules
----------------------------------------
----------------------------------------
Minimal simulation completed.
